## The Python web scraping notebook 5: Player Capping Details

### https://www.espncricinfo.com/
* Mens cricket player Data for all capped players for top 10 countries

In [5]:
# Importing the libraries
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

In [13]:
# Country to country ID mapping

main_link = 'https://www.espncricinfo.com/ci/content/player/index.html'
response = requests.get(main_link)
html_soup = BeautifulSoup(response.text, 'html.parser')
text = str(html_soup.find_all("li", class_=['selectedtab',"ctrytab"]))
html_soup.find_all("li", class_=['selectedtab',"ctrytab"])

country_id = re.findall(r'(?<=country=)[0-9]+(?=")', text)
#print(country_id)
country_name = re.findall(r'(?<=">)[a-zA-Z ]+(?=<\/a)', text)
#print(country_name)

country_data = pd.DataFrame({'Id' : country_id,
                            'Country_Name' : country_name
                                })
country_data.to_csv("Country Data.csv")
country_data

Format_dict = {'1':'Test', '2':'ODI', '3':'T20'}

### Getting the Latest Team rankings

In [16]:
# Test links for webscraping
# Country list 1-10
# Classes 1 = test, 2= odi, 3= t20
Capping_data = pd.DataFrame()
for i in range(len(country_data)):
    for j in range(1,4):
        link = 'https://www.espncricinfo.com/ci/content/player/caps.html?country=%s;class=%s' % (country_data.Id[i],j)
        print(link)
        data = extract_data(link,i,j)
        #print(data)
        Capping_data = pd.concat([Capping_data,data],ignore_index = True)

https://www.espncricinfo.com/ci/content/player/caps.html?country=2;class=1
458
458
https://www.espncricinfo.com/ci/content/player/caps.html?country=2;class=2
229
229
https://www.espncricinfo.com/ci/content/player/caps.html?country=2;class=3
94
94
https://www.espncricinfo.com/ci/content/player/caps.html?country=25;class=1
96
96
https://www.espncricinfo.com/ci/content/player/caps.html?country=25;class=2
133
133
https://www.espncricinfo.com/ci/content/player/caps.html?country=25;class=3
68
68
https://www.espncricinfo.com/ci/content/player/caps.html?country=1;class=1
695
695
https://www.espncricinfo.com/ci/content/player/caps.html?country=1;class=2
257
257
https://www.espncricinfo.com/ci/content/player/caps.html?country=1;class=3
91
91
https://www.espncricinfo.com/ci/content/player/caps.html?country=6;class=1
296
296
https://www.espncricinfo.com/ci/content/player/caps.html?country=6;class=2
231
231
https://www.espncricinfo.com/ci/content/player/caps.html?country=6;class=3
82
82
https://www

In [15]:
# For Player Data
def extract_data(link,i,j):
    response = requests.get(link)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    text = str(html_soup.find_all("li", class_="sep"))
    #html_soup.find_all("li")
    player_name = re.findall(r'(?<=le;">)[A-Za-z -.0-9]+(?=<\/a)', text)
    player_first_match = re.findall(r'(?<=html">)[A-Za-z -.,0-9\/]+(?=<\/a)', text)
    player_id = re.findall(r'(?<=player/)[A-Za-z -.,0-9\/]+(?=\.html)', text)

    print(len(player_name))
    print(len(player_first_match))


    Capped_players = pd.DataFrame({
                                'Player Name' : player_name,
                                  'Player First Match' : player_first_match,
                                    'Player_cricinfo_id' : player_id
                                    })

    Capped_players['Format'] = Format_dict[str(j)]
    Capped_players['Format_id'] = str(j)
    Capped_players['Country'] = country_data['Country_Name'][i]
    Capped_players['Country_id'] = country_data['Id'][i]
    #print(Capped_players)
    return(Capped_players)

In [18]:
# Writing out

Capping_data.to_csv("Capped_players.csv",index=False)